In [2]:
from datetime import datetime, timedelta
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import time

In [3]:
def get_tickers(file=r"Stock names.txt"):
    with open(file) as f:
        tickers = f.read().split("\n")
    return [ticker for ticker in tickers if "." not in ticker]

In [4]:
def get_all_tickers():
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]
    
    # with open('tickers.txt', 'w') as file:
    #     for ticker in tickers:
    #         file.write(ticker + '\n')
# get_all_tickers()[:10]

In [5]:
def get_info(tickers_list):
    df = yf.download(tickers_list, period="1y")

    highs = df['High'].max()
    closes = df['Close'].iloc[-1]
    pct_to_52 = ((highs - closes) / highs) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': closes, '52wk High': highs, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high})

    return new_df

In [6]:
s = time.time()
df = get_info(get_all_tickers())
print(time.time() - s)
cop = df.copy()

[*********************100%***********************]  10607 of 10607 completed

75 Failed downloads:
- GJUL: No data found, symbol may be delisted
- ZIEXT: No data found, symbol may be delisted
- ZBZZT: Period '1y' is invalid, must be one of ['1d', '5d']
- SVIIR: Period '1y' is invalid, must be one of ['1d', '5d']
- ZEXIT: No data found, symbol may be delisted
- CSLMR: Period '1y' is invalid, must be one of ['1d', '5d']
- ASCBR: Period '1y' is invalid, must be one of ['1d', '5d']
- ATEST: Period '1y' is invalid, must be one of ['1d', '5d']
- MSSAR: Period '1y' is invalid, must be one of ['1d', '5d']
- BREZR: Period '1y' is invalid, must be one of ['1d', '5d']
- GFGDR: Period '1y' is invalid, must be one of ['1d', '5d']
- ZVZZT: Period '1y' is invalid, must be one of ['1d', '5d']
- KACLR: Period '1y' is invalid, must be one of ['1d', '5d']
- CETUR: Period '1y' is invalid, must be one of ['1d', '5d']
- GBBKR: Period '1y' is invalid, must be one of ['1d', '5d']
- ATMCR: Period '1y' is inval

In [69]:
df = df.dropna(subset=['Close'])
vol_thresh = 10000
df = df[df['Volume'] >= vol_thresh]
time_thresh = datetime.now() - timedelta(days=60)
df = df[df["doh"] < time_thresh]
df.sort_values(by='Percent from 52', ascending=True)

,Close,52wk High,Percent from 52,Volume,doh
AGNCN,25.549999,25.690001,0.544964,15800.0,2023-03-06
AIO,18.900000,19.110001,1.098906,110500.0,2022-08-15
ALIT,10.050000,10.185000,1.325481,3521800.0,2023-02-22
ACWV,98.709999,100.410004,1.693063,559300.0,2022-08-16
ADME,37.000999,37.700001,1.854115,101400.0,2022-08-16
...,...,...,...,...,...
AKAN,0.690000,15.500000,95.548387,98600.0,2022-08-09
AKLI,1.160000,37.581001,96.913334,41000.0,2022-08-22
ADTX,0.429000,28.490000,98.494209,71700.0,2022-09-14
AGFY,2.740000,416.000000,99.341346,53500.0,2022-10-17


In [16]:
def get_data(tickers_list):
    return yf.download(tickers_list, period="1y")
    

In [78]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df


In [ ]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

In [79]:
df = get_info_test(data)
cop = df.copy()
cop

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,117.00,160.26,26.99,3098500.00,2022-12-13,NaN,NaN,160.26,2022-12-13
AA,34.09,58.24,41.46,8763950.00,2022-08-26,NaN,NaN,58.24,2022-08-26
AAA,24.58,24.73,0.58,3550.00,2023-02-22,NaN,NaN,24.82,2023-07-10
AAAU,19.08,20.43,6.61,893750.00,2023-05-04,NaN,NaN,20.43,2023-05-04
AAC,10.57,10.58,0.09,309650.00,2023-06-30,NaN,NaN,10.60,2023-07-18
...,...,...,...,...,...,...,...,...,...
ZXIET,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
ZXZZT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
ZYME,8.26,10.80,23.52,484550.00,2023-01-19,NaN,NaN,10.80,2023-01-19
ZYNE,0.35,1.39,74.82,225900.00,2022-08-08,NaN,NaN,1.39,2022-08-08


In [81]:
# Filtering and sorting

df = cop.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=9)  # How many days ago it broke
vol_thresh = 10000

df = df[df["Old 52wk High"] != df["high52"]]
# df = df[df['Old Volume'] >= vol_thresh]
df = df[df["doh"] < time_thresh]
df = df[df["date of break"] < break_time_thresh]
df["diff"] = datetime.now() - df["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df.sort_values(by='Volume', ascending=False).head(20)

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break,diff
MARA,15.67,18.88,17.00,29030000.00,2022-08-11,NaN,NaN,19.88,2023-07-14,10 days 12:38:52.331717
SNAP,11.68,12.94,9.74,20912000.00,2022-09-12,NaN,NaN,13.89,2023-07-13,11 days 12:38:52.331717
AUR,2.80,3.27,14.37,14424950.00,2022-08-11,NaN,NaN,3.54,2023-07-12,12 days 12:38:52.331717
BWA,44.95,45.02,0.15,1709850.00,2023-03-03,NaN,NaN,47.05,2023-07-13,11 days 12:38:52.331717
ROVR,4.84,5.26,7.98,1527550.00,2022-11-08,NaN,NaN,5.29,2023-07-12,12 days 12:38:52.331717
DFAE,23.42,24.36,3.86,1458350.00,2023-01-26,NaN,NaN,24.41,2023-07-13,11 days 12:38:52.331717
HLT,148.32,152.89,2.99,1438900.00,2023-02-15,NaN,NaN,155.02,2023-07-11,13 days 12:38:52.331717
INMD,38.07,41.84,9.01,1278750.00,2023-02-16,NaN,NaN,46.46,2023-07-13,11 days 12:38:52.331717
VBLT,0.26,0.31,15.87,1103500.00,2022-07-25,NaN,NaN,0.32,2023-07-13,11 days 12:38:52.331717
MGNI,13.30,14.21,6.40,971050.00,2023-02-22,NaN,NaN,15.73,2023-07-13,11 days 12:38:52.331717


In [61]:
data_cop
# cop

Adj Close                                                       \
                   A    AA   AAA  AAAU   AAC AACG  AACI AACIU AACIW  AACT   
Date                                                                        
2022-07-25    124.26 46.80 24.26 17.07  9.87 1.54  9.89 10.01   NaN   NaN   
2022-07-26    125.17 46.99 24.26 17.06  9.87 1.54  9.91 10.01   NaN   NaN   
2022-07-27    127.71 47.75 24.24 17.22  9.86 1.53  9.91 10.01   NaN   NaN   
2022-07-28    131.11 49.08 24.29 17.43  9.87 1.55  9.91 10.01   NaN   NaN   
2022-07-29    134.10 50.89 24.30 17.49  9.87 1.64  9.92 10.01   NaN   NaN   
...              ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   
2023-07-18    119.56 35.04 24.64 19.61 10.57 1.44 10.51 10.53   NaN 10.16   
2023-07-19    123.08 34.84 24.65 19.61 10.58 1.44 10.54 10.53   NaN 10.16   
2023-07-20    124.55 32.68 24.67 19.51 10.56 1.40 10.52 10.53   NaN 10.17   
2023-07-21    127.62 33.38 24.72 19.45 10.56 1.39 10.51 10.53  0.07 10.17   
2023-07-23       NaN   NaN   NaN   NaN   NaN  NaN   NaN   NaN   NaN   NaN   

            ... Volume                                                      \
            ...    ZVV ZVZZC ZVZZT        ZWS ZWZZT ZXIET ZXZZT       ZYME   
Date        ...                                                              
2022-07-25  ...    NaN   NaN   NaN 1053000.00   NaN   NaN   NaN 1003600.00   
2022-07-26  ...    NaN   NaN   NaN 1696900.00   NaN   NaN   NaN  646900.00   
2022-07-27  ...    NaN   NaN   NaN 4468500.00   NaN   NaN   NaN  391100.00   
2022-07-28  ...    NaN   NaN   NaN 1512400.00   NaN   NaN   NaN  475700.00   
2022-07-29  ...    NaN   NaN   NaN 1893800.00   NaN   NaN   NaN  723200.00   
...         ...    ...   ...   ...        ...   ...   ...   ...        ...   
2023-07-18  ...    NaN   NaN   NaN  608200.00   NaN   NaN   NaN  363100.00   
2023-07-19  ...    NaN   NaN   NaN  786500.00   NaN   NaN   NaN  500100.00   
2023-07-20  ...    NaN   NaN   NaN  665900.00   NaN   NaN   NaN  335500.00   
2023-07-21  ...    NaN   NaN   NaN 3603600.00   NaN   NaN   NaN  633600.00   
2023-07-23  ...    NaN   NaN   NaN        NaN   NaN   NaN   NaN        NaN   

                                
                ZYNE      ZYXI  
Date                            
2022-07-25 271100.00 140100.00  
2022-07-26 476400.00 133000.00  
2022-07-27 274100.00 192700.00  
2022-07-28 331500.00 274500.00  
2022-07-29 189200.00 594200.00  
...              ...       ...  
2023-07-18 133200.00 142000.00  
2023-07-19 344700.00 114400.00  
2023-07-20 374600.00 105400.00  
2023-07-21  77200.00 129200.00  
2023-07-23       NaN       NaN  

[251 rows x 63642 columns]